# Xtream AI assignement - Challenge 1 - Model Selection

## Purpose
The outcome of this notebook will be a model that predicts a gem's worth based on its characteristics.

## WIP - improvements

Notable TODOs:

- NORMALIZE
- consider weighted loss function
- insert clas weights during training
- xboost regressor
- fissa randpm state


## Library import

In [1]:
import pandas as pd

## Import data

In [3]:
## Load the data
df = pd.read_csv('../datasets/diamonds/clean_data.csv')
df

,depth,table,cut_enc,color_enc,clarity_enc,log_price,log_carat
0,62.0,55.0,4,18,1,8.462315,0.095310
1,62.6,56.0,4,18,2,8.767796,0.254642
2,61.1,58.0,3,17,2,8.614320,0.182322
3,60.9,56.0,4,20,2,9.079092,0.405465
4,61.7,57.0,2,20,3,8.410276,-0.105361
...,...,...,...,...,...,...,...
4407,62.3,53.3,4,18,5,6.723832,-0.967584
4408,61.3,59.0,3,19,5,6.831954,-1.108663
4409,62.1,56.0,4,16,5,8.696176,0.223144
4410,62.9,58.0,3,20,3,6.687109,-1.171183


In [5]:
column_name_mapping = {'cut_enc': 'cut', 'color_enc': 'color','clarity_enc':'clarity', 'log_price':'price','log_carat':'carat' }
df = df.rename(columns=column_name_mapping)
df

,depth,table,cut,color,clarity,price,carat
0,62.0,55.0,4,18,1,8.462315,0.095310
1,62.6,56.0,4,18,2,8.767796,0.254642
2,61.1,58.0,3,17,2,8.614320,0.182322
3,60.9,56.0,4,20,2,9.079092,0.405465
4,61.7,57.0,2,20,3,8.410276,-0.105361
...,...,...,...,...,...,...,...
4407,62.3,53.3,4,18,5,6.723832,-0.967584
4408,61.3,59.0,3,19,5,6.831954,-1.108663
4409,62.1,56.0,4,16,5,8.696176,0.223144
4410,62.9,58.0,3,20,3,6.687109,-1.171183


## normalize

## split train/test


## EVALUATION METRICS
RMSE
plotta grafico con actual vs predicted e riga bisettrice
sulla x e la y i prezzi attuali e predetti

Dataset split to test the performances of the model:

- 80% training
- 20% test to perform an evaluation that simulated realworld scenarios

for each model you choose to use, correlate the choice based on the data you saw already